# **Task:**
### **Method for local weights and consistency measure computing:** Eigenvector method(EM), CR
### **Method for global weights computing:** Ideal

# **Importing all necessary libraries**

In [ ]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# **Preparing models (alternatives)**

For this task chosen dataset from kaggle with information about heart diseases: https://www.kaggle.com/datasets/yasserh/heart-disease-dataset.

## **Processing data**

In [ ]:
df=pd.read_csv('/content/heart.csv')
display(df)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [ ]:
y=df['target'].values
X=df.drop(['target'],axis=1).values
print(y[:5])
print(X[:5])

[1 1 1 1 1]
[[ 63.    1.    3.  145.  233.    1.    0.  150.    0.    2.3   0.    0.
    1. ]
 [ 37.    1.    2.  130.  250.    0.    1.  187.    0.    3.5   0.    0.
    2. ]
 [ 41.    0.    1.  130.  204.    0.    0.  172.    0.    1.4   2.    0.
    2. ]
 [ 56.    1.    1.  120.  236.    0.    1.  178.    0.    0.8   2.    0.
    2. ]
 [ 57.    0.    0.  120.  354.    0.    1.  163.    1.    0.6   2.    0.
    2. ]]


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.9)
print(X_train.shape)
print(X_test.shape)

(272, 13)
(31, 13)


## **Training models**

In [ ]:
f1s=[]
aucs=[]
models=[]

In [ ]:
model=LogisticRegression().fit(X_train,y_train)
f1s.append(f1_score(y_test,model.predict(X_test)))
aucs.append(roc_auc_score(y_test,model.predict(X_test)))
models.append('Logistic Regression')

In [ ]:
model=SVC().fit(X_train,y_train)
f1s.append(f1_score(y_test,model.predict(X_test)))
aucs.append(roc_auc_score(y_test,model.predict(X_test)))
models.append('SVM')

In [ ]:
model=KNeighborsClassifier().fit(X_train,y_train)
f1s.append(f1_score(y_test,model.predict(X_test)))
aucs.append(roc_auc_score(y_test,model.predict(X_test)))
models.append('KNN')

In [ ]:
model=GradientBoostingClassifier().fit(X_train,y_train)
f1s.append(f1_score(y_test,model.predict(X_test)))
aucs.append(roc_auc_score(y_test,model.predict(X_test)))
models.append('Gradient Boosting')

In [ ]:
metrics=np.array([f1s,aucs]).T
summary=pd.DataFrame(index=models,data=metrics,columns=['F1','AUC'])
display(summary)

,F1,AUC
Logistic Regression,0.967742,0.968750
SVM,0.742857,0.706250
KNN,0.787879,0.772917
Gradient Boosting,0.933333,0.937500


# **Computing local weights**

As two criterions f1-score and auc-score will be taken. In this case consistency matrix don't really have to be computed, and corresponding scores can be taken as non-normalized local weights of corresponding alternatives with corresponding criterion. Also another criterion, pretty much subjective, was chosen, which already requires building consistency matrix. To be more exact, as the third criterion was choosen possibility of flexible model's customization(like in svm we can choose different kernels, which dramatically will affect model's permormance, in gradient boosting we can set number of models into ensemble, max depth of trees, etc).

## **Building pairwise comparison matrix**

In [ ]:
D_settings=np.array([[1,1/5,1/2,1/7],
                     [5,  1,  4,1/3],
                     [2,1/4,  1,1/7],
                     [7,  3,  7,  1]])

## **Using power method for computation largest eigenvalue with corresponding eigenvector**

In [ ]:
eps=1e-3
lmbd_=0
x=np.array([[1,0,0,0]]).T
Dx=D_settings.dot(x)
lmbd=np.sum(np.fabs(Dx))
print('Iteration 1:')
print(f'eigvalue={lmbd}')
print(f'eigvector={Dx}')
i=2
print('\n')
while(np.fabs(lmbd-lmbd_)>eps):
    lmbd_=lmbd
    x=Dx/lmbd
    Dx=D_settings.dot(x)
    lmbd=Dx[0]/x[0]
    print(f'Iteration {i}:')
    print(f'eigvalue={lmbd}')
    print(f'eigvector={Dx}')
    i+=1
    print('\n')

Iteration 1:
eigvalue=15.0
eigvector=[[1.]
 [5.]
 [2.]
 [7.]]


Iteration 2:
eigvalue=[4.]
eigvector=[[0.26666667]
 [1.35555556]
 [0.41666667]
 [2.86666667]]


Iteration 3:
eigvalue=[4.33363095]
eigvector=[[0.28890873]
 [1.32777778]
 [0.42460317]
 [2.92916667]]


Iteration 4:
eigvalue=[4.10239681]
eigvector=[[0.27349312]
 [1.25694206]
 [0.40446856]
 [2.74759975]]


Iteration 5:
eigvalue=[4.09381435]
eigvector=[[0.27292096]
 [1.25734996]
 [0.40420384]
 [2.7457504 ]]


Iteration 6:
eigvalue=[4.0991463]
eigvector=[[0.27327642]
 [1.25897753]
 [0.40466742]
 [2.74992292]]


Iteration 7:
eigvalue=[4.09933564]
eigvector=[[0.27328904]
 [1.25896221]
 [0.40467226]
 [2.74995367]]




## **Recieving non-normalized vector of weights and largest eigenvalue**

In [ ]:
eig_val=lmbd[0]
print(eig_val)
eig_vector=Dx
print(eig_vector)

4.099335642044864
[[0.27328904]
 [1.25896221]
 [0.40467226]
 [2.74995367]]


## **Checking pairwise comparison matrix on consistency using CR**

In [ ]:
ci=(eig_val-4)/(4-1)
mrci=0.89
cr=ci/mrci
print(f'cr={cr}')
threshold=0.08
if cr<threshold:
    print('Consistent')
else:
    print('Non-Consistent')

cr=0.03720436031642859
Consistent


As it can be seen, CR less than threshold value 0.08, so our matrix is acceptably non-consistent.

# **Computing global weights**

## **Defining weights for different criterions**

In [ ]:
weights_criterion=[0.45,0.4,0.15]

## **Building matrix of non-normalized weights for different criterions**

In [ ]:
weights_alternative=np.array([f1s,aucs,Dx.T[0]]).T
print(weights_alternative)

[[0.96774194 0.96875    0.27328904]
 [0.74285714 0.70625    1.25896221]
 [0.78787879 0.77291667 0.40467226]
 [0.93333333 0.9375     2.74995367]]


## **Computing non-normalized global weights**

In [ ]:
weights_glob=[]
for i in range(4):
    wi_glob=0
    for j in range(3):
        rij=weights_alternative[i,j]/max(weights_alternative[:,j])
        wi_glob+=weights_criterion[j]*rij
    weights_glob.append(wi_glob)
print(weights_glob)

[0.8649069262095146, 0.705713297758537, 0.7075768252892635, 0.9710967741935485]
